::::
:::{thebe-button}
:::
::::

# Plot tracks

Compare experimental bubble histories to bubble history correlations. Bubble detection and linking performed by Trackpy, an implementation of the Crocker-Grier algorithm {cite}`allanTrackpy2018,crockerMethodsDigitalVideo1996`.


In [ ]:
from boilercv_docs.nbs import init

paths = init()

from boilercv_docs.nbs import HIDE
from matplotlib.figure import Figure
from matplotlib.pyplot import subplot_mosaic, subplots
from numpy import diff, log10, logspace
from pandas import DataFrame, Series, read_hdf
from seaborn import lineplot, move_legend, scatterplot

from boilercv.correlations import (
    dimensionless_bubble_diameter_florschuetz,
    dimensionless_bubble_diameter_tang,
    dimensionless_bubble_diameter_yuan,
    fourier,
    jakob,
    kinematic_viscosity,
    prandtl,
    reynolds,
    thermal_diffusivity,
)
from boilercv.data.sets import get_dataset
from boilercv.experiments.e230920_subcool import (
    GBC,
    M_TO_MM,
    OBJECTS,
    THERMAL_DATA,
    TRACKS,
    Col,
    get_cat_colorbar,
    plot_composite_da,
    transform_cols,
)
from boilercv.experiments.e240215_plotting import cool, warm12
from boilercv.images import scale_bool

S = Series


TIME = "2023-09-20T17:14:18"
"""Timestamp of the trial to be analyzed."""
FIGURES: list[Figure] = []
"""Notebook figures available for export."""

# Track tuning
YPX_SURFACE_THRESHOLD = 400
"""Vertical position of bubble centroids considered attached to the surface."""
YPX_DEPARTURE_THRESHOLD = 420
"""Vertical position of bubble centroids considered to have departed the surface."""
MINIMUM_LIFETIME = 0.010  # (s)
"""Minimum bubble lifetime to consider."""

# Physical parameters
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3

# Plotting
FRAME_INTERVAL = 50
"""Frame interval for plotting."""
CORRELATIONS_PALETTE = warm12
"""For plotting one approach."""
TRACKS_PALETTE = cool
"""For plotting the other approach."""
MAX_FOURIER = 0.02
"""Maximum Fourier number to plot."""
MAX_DIMENSIONLESS_BUBBLE_DIAMETER = 1.05
"""Maximum dimensionless bubble diameter to plot."""
TRACKS_ALPHA = 0.3
"""Transparency of the tracks."""
TRACKS_SIZE = 10
"""Size of the tracks."""


def get_init(ser: S) -> float:
    """Get initial value of a series."""
    return ser.head(len(ser) // 100).mean()


HIDE

In [ ]:
path_time = TIME.replace(":", "-")
video = scale_bool(
    get_dataset(path_time, stage="filled", frame=slice(0, None, FRAME_INTERVAL))[
        "video"
    ]
)
frametime = diff(video.time.values).mean() / FRAME_INTERVAL
objects: DataFrame = read_hdf((OBJECTS / f"objects_{path_time}").with_suffix(".h5"))  # type: ignore
subcooling = read_hdf(THERMAL_DATA).subcool[TIME]
minimum_frame_lifetime = int(MINIMUM_LIFETIME // frametime)
raw_tracks: DataFrame = read_hdf((TRACKS / f"tracks_{path_time}").with_suffix(".h5"))  # type: ignore

HIDE

## History of long-lived bubbles

The paths taken by long-lived bubbles are shown in **Figure&NonBreakingSpace;2**. Two active nucleation sites are responsible for all bubbles produced, and bubbles departing from each nucleation site take one of a few predominant paths during the short period of observation.


In [ ]:
figure, ax = subplots()
FIGURES.append(figure)
plot_composite_da(video, ax)
long_lived_objects = raw_tracks.query(f"frame_lifetime > {minimum_frame_lifetime}")
N = long_lived_objects["bubble"].nunique()
palette, data = get_cat_colorbar(
    ax,
    palette=TRACKS_PALETTE,
    data=long_lived_objects.pipe(
        transform_cols,
        [
            hue := Col("bubble", "Individual bubble"),
            x := Col("x_px", "x", "px"),
            y := Col("y_px", "y", "px"),
        ],
    ),
    col=hue.new,
)
scatterplot(
    ax=ax,
    edgecolor="none",
    s=10,
    x=x.new,
    y=y.new,
    hue=hue.new,
    legend=False,  # type: ignore  # pyright 1.1.333
    palette=palette,
    data=data,
)

HIDE

**Figure&NonBreakingSpace;1**: Long-lived bubble tracks  
Bubble tracks indicated by the positions of detected centroids over time.

## Aligning bubble departures

Exclude bubbles that did not originate from the boiling surface, or that had already departed the surface at the time of recording. Consider a bubble to have departed the surface when its centroid crosses a departure threshold which is about one average bubble diameter above the boiling surface. Define the origin for time of departure for each bubble in this fashion. The resulting time history in **Figure&NonBreakingSpace;2** shows bubble depth, velocity, and diameter for the remainder of its visible lifetime.

Most bubbles rise and collapse at similar rates. Two bubbles rise slower than the rest, but seem to collapse at about the same rate as others.


In [ ]:
departing_long_lived_objects = (
    # Find rows corresponding to stagnant or invalid bubbles
    long_lived_objects.groupby("bubble", **GBC)
    .apply(
        # Don't assign any other columns until invalid rows have been filtered out
        lambda df: df.assign(
            yinit_px=lambda df: df["y_px"].pipe(get_init),
            # Initial y position is close to the surface
            began=lambda df: df["yinit_px"] > YPX_SURFACE_THRESHOLD,
            # When the bubble gets far enough away from the surface
            departed=lambda df: df["y_px"] < YPX_DEPARTURE_THRESHOLD,
        )
    )
    # Filter out invalid rows and drop the columns used to determine validity
    .pipe(lambda df: df[df["began"] & df["departed"]])
    .drop(columns=["began", "departed"])
    # Groupby again after filtering out invalid rows
    .groupby("bubble", **GBC)
    # Now columns that depend on the initial row (*.iat[0]) can be assigned
    .apply(
        lambda df: df.assign(
            frame=lambda df: df["frame"] - df["frame"].pipe(get_init),
            time=lambda df: df["time"] - df["time"].pipe(get_init),
            frame_lifetime=lambda df: df["frame"].iat[-1] - df["frame"].pipe(get_init),
            lifetime=lambda df: df["frame_lifetime"] * frametime,
            y_init=lambda df: df["y"].pipe(get_init),
            x_init=lambda df: df["x"].pipe(get_init),
            init_diameter=lambda df: df["diameter"].pipe(get_init),
            dy_init=lambda df: df["dy"].pipe(get_init),
            dy_init_px=lambda df: df["dy"].pipe(get_init),
            max_diameter=lambda df: df["diameter"].max(),
        )
    )
)
cols = [
    hue := Col("bubble", "Individual bubble"),
    x := Col("time", "Time after departure", "s"),
    y := Col("y", "Depth", **M_TO_MM),
    v := Col("dy", "Velocity", "m/s", "mm/s"),
    d := Col("diameter", "Diameter", **M_TO_MM),
]
figure, axs = subplot_mosaic([[y.new], [v.new], [d.new]])
FIGURES.append(figure)
figure.set_size_inches(6, 10)
for plot, ax in axs.items():
    palette, data = get_cat_colorbar(
        ax,
        hue.new,
        TRACKS_PALETTE,
        departing_long_lived_objects.pipe(transform_cols, cols),
    )
    scatterplot(
        ax=ax,
        edgecolor="none",
        s=TRACKS_SIZE,
        alpha=TRACKS_ALPHA,
        x=x.new,
        y=plot,
        hue=hue.new,
        legend=False,  # type: ignore  # pyright 1.1.333
        palette=palette,
        data=data,
    )

**Figure&NonBreakingSpace;2**: Time history of long-lived bubbles Bubble depth, velocity, and diameter plotted over time.

Histograms of individual bubble statistics are shown in **Figure&NonBreakingSpace;3**. Because no bubbles completely collapse, bubble lifetimes correspond to the duration of time between their departure from the boiling surface and rising past the upper limit of the camera viewpoint. The maximum bubble diameter is about 3&NonBreakingSpace;mm.

The bimodal distribution of initial $x$ positions is also evident, corresponding to two active nucleation sites. Initial bubble velocity at departure tends to be about 250&NonBreakingSpace;mm/s.


In [ ]:
liquid_thermal_diffusivity = thermal_diffusivity(
    thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    density=LIQUID_DENSITY,
    isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
)
bubble_fourier_experimental = fourier(
    initial_bubble_diameter=departing_long_lived_objects["init_diameter"],  # type: ignore
    liquid_thermal_diffusivity=liquid_thermal_diffusivity,
    time=departing_long_lived_objects["time"],  # type: ignore
)
tracks = departing_long_lived_objects.assign(**{
    "Bubble Fourier number": bubble_fourier_experimental,
    "Dimensionless bubble diameter": (lambda df: df["diameter"] / df["init_diameter"]),
})
figure, ax = subplots()
FIGURES.append(figure)
(
    tracks.pipe(
        transform_cols,
        [
            Col("bubble"),
            Col("lifetime", "Lifetime", "s"),
            Col("max_diameter", r"$d_{max}$", **M_TO_MM),
            Col("init_diameter", r"$d_{b0}$", **M_TO_MM),
            Col("y_init", r"$y_{b0}$", **M_TO_MM),
            Col("x_init", r"$x_{b0}$", **M_TO_MM),
            Col("dy_init", r"$v_{y0}$", old_unit="m/s", new_unit="mm/s", scale=1000),
        ],
    )
    .groupby("bubble", **GBC)
    .mean()
    .set_index("bubble")
    .hist(ax=ax)
)

HIDE

**Figure&NonBreakingSpace;3**: Histograms of individual bubble statistics  
Shows bubble lifetime, maximum diameter, and bubble properties at departure.


# Correlations

See {eq}`eq_dimensionless_bubble_diameter_florschuetz` and {eq}`eq_dimensionless_bubble_diameter_yuan`.

In [ ]:
object_averages = (
    departing_long_lived_objects.set_index("bubble")
    .groupby("bubble", **GBC)
    .mean()
    .mean()
)
liquid_kinematic_viscosity = kinematic_viscosity(
    density=LIQUID_DENSITY, dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY
)
bubble_initial_reynolds = reynolds(
    velocity=abs(object_averages["dy_init"]),
    characteristic_length=object_averages["init_diameter"],
    kinematic_viscosity=liquid_kinematic_viscosity,
)
liquid_prandtl = prandtl(
    dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
    isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
)
bubble_jakob = jakob(
    liquid_density=LIQUID_DENSITY,
    vapor_density=VAPOR_DENSITY,
    liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    subcooling=subcooling,
    latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
)
bubble_fourier_smooth = logspace(
    stop=log10(MAX_FOURIER), start=log10(MAX_FOURIER) - 4, num=int(1e4)
)
figure, ax = subplots()
FIGURES.append(figure)
ax.set_xlim(0, MAX_FOURIER)
ax.set_ylim(0, MAX_DIMENSIONLESS_BUBBLE_DIAMETER)
ax.plot(label=f"{subcooling:.2f} K")
lineplot(
    ax=ax,
    data=(
        data := DataFrame(index=bubble_fourier_smooth)
        .assign(**{
            f"{subcooling:.2f} K subcooling": None,
            "G1, Florshuetz and Chao (1965)": dimensionless_bubble_diameter_florschuetz(
                jakob=bubble_jakob, fourier=bubble_fourier_smooth
            ),
            "G3, Tang et al. (2016)": dimensionless_bubble_diameter_tang(
                bubble_initial_reynolds=bubble_initial_reynolds,
                liquid_prandtl=liquid_prandtl,
                bubble_jakob=bubble_jakob,
                bubble_fourier=bubble_fourier_smooth,
            ),
            "G4, Yuan et al. (2009)": dimensionless_bubble_diameter_yuan(
                bubble_initial_reynolds=bubble_initial_reynolds,
                liquid_prandtl=liquid_prandtl,
                bubble_jakob=bubble_jakob,
                bubble_fourier=bubble_fourier_smooth,
            ),
        })
        .where(lambda s: s > 0.0)
    ),
    palette=[(0, 0, 0, 0), *CORRELATIONS_PALETTE],
)
move_legend(ax, "lower center", bbox_to_anchor=(0.5, 1))
ax.get_legend().get_texts()[0].set_weight("bold")
palette, data = get_cat_colorbar(
    ax,
    palette=TRACKS_PALETTE,
    data=tracks.pipe(
        transform_cols,
        [
            hue := Col("bubble", "Individual bubble"),
            x := Col("Bubble Fourier number"),
            y := Col("Dimensionless bubble diameter"),
        ],
    ),
    col=hue.new,
    alpha=TRACKS_ALPHA,
)
scatterplot(
    ax=ax,
    s=TRACKS_SIZE,
    alpha=TRACKS_ALPHA,
    x=x.new,
    y=y.new,
    hue=hue.new,
    palette=TRACKS_PALETTE,
    legend=False,  # type: ignore  # pyright 1.1.333
    data=data,
)

HIDE

**Figure&NonBreakingSpace;5**: Comparison of bubble histories to correlations

Two correlations are shown. The early bubble history follows that of the analytical correlation by Florshuetz and Chao (1965), while the late bubble history follows that of Yuan et al (2009).
